In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2 as cv
import random
import os
from shutil import copyfile
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
def visualize_random(folder):
    plt.figure(figsize=(20,10))
    images = os.listdir(folder)
    for i in range(10):
        img = random.choice([x for x in images])
        fig = plt.subplot(5, 5, i + 1)
        fig.axis('off')
        img = mpimg.imread(os.path.join(folder, img))
        fig.imshow(img)

In [3]:
try:
    os.mkdir('total')
    for dir in ['training', 'validation']:
        path = os.path.join('total', dir)
        os.mkdir(path)
        for dir in ['cat', 'dog']:
            n_path = os.path.join(path, dir)
            os.mkdir(n_path)
except Exception:
    print('Existed!')

In [4]:
cats = [x for x in os.listdir('train') if 'cat' in x]
dogs = [x for x in os.listdir('train') if 'dog' in x]
cat_test = random.sample(cats, int(0.2 * len(cats)))
dog_test = random.sample(dogs, int(0.2 * len(dogs)))

In [5]:
for img in os.listdir('train'):
    src_img = os.path.join('train', img)
    if 'cat' in img:
        if img in cat_test:
            dst_img = os.path.join('total/validation/cat', img)
        else:
            dst_img = os.path.join('total/training/cat', img)
    elif 'dog' in img:
        if img in dog_test:
            dst_img = os.path.join('total/validation/dog', img)
        else:
            dst_img = os.path.join('total/training/dog', img)
    copyfile(src_img, dst_img)

In [6]:
gen_train = ImageDataGenerator(rescale=1/255,
                               rotation_range=40, 
                               width_shift_range=0.2,
                               height_shift_range=0.2,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True, 
                               fill_mode='nearest')
gen_valid = ImageDataGenerator(rescale=1/255)

img_train = gen_train.flow_from_directory('total/training', target_size=(150, 150), batch_size=32, class_mode='binary')
img_valid = gen_valid.flow_from_directory('total/validation', target_size=(150, 150), batch_size=32, class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [7]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = Sequential([Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
                        MaxPooling2D(2, 2),
                        Conv2D(64, (3,3), activation='relu'),
                        MaxPooling2D(2,2),
                        Conv2D(128, (3,3), activation='relu'),
                        MaxPooling2D(2,2),
                        Flatten(),
                        Dense(128, activation='relu'),
                        Dense(1, activation='sigmoid')])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
            
model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
____________________

In [8]:
history = model.fit(img_train, 
                    epochs=20,
                    validation_data=img_valid)

Epoch 1/20
625/625 [==============================] - 580s 923ms/step - loss: 0.7032 - accuracy: 0.5254 - val_loss: 0.6838 - val_accuracy: 0.5560
Epoch 2/20
625/625 [==============================] - 579s 927ms/step - loss: 0.6559 - accuracy: 0.6166 - val_loss: 0.5586 - val_accuracy: 0.7186
Epoch 3/20
625/625 [==============================] - 573s 917ms/step - loss: 0.5905 - accuracy: 0.6825 - val_loss: 0.4953 - val_accuracy: 0.7582
Epoch 4/20
625/625 [==============================] - 572s 915ms/step - loss: 0.5587 - accuracy: 0.7118 - val_loss: 0.4837 - val_accuracy: 0.7688
Epoch 5/20
625/625 [==============================] - 570s 912ms/step - loss: 0.5368 - accuracy: 0.7258 - val_loss: 0.4679 - val_accuracy: 0.7854
Epoch 6/20
625/625 [==============================] - 570s 912ms/step - loss: 0.5127 - accuracy: 0.7493 - val_loss: 0.4559 - val_accuracy: 0.7820
Epoch 7/20
625/625 [==============================] - 578s 924ms/step - loss: 0.4889 - accuracy: 0.7626 - val_loss: 0.4627 -

In [9]:
df_hist = pd.DataFrame(history.history)
df_hist

,loss,accuracy,val_loss,val_accuracy
0,0.687215,0.55320,0.683762,0.5560
1,0.641075,0.63730,0.558613,0.7186
2,0.580951,0.69605,0.495319,0.7582
3,0.548195,0.71750,0.483681,0.7688
4,0.533156,0.72925,0.467870,0.7854
5,0.505974,0.75235,0.455888,0.7820
6,0.488756,0.76280,0.462720,0.7732
7,0.467196,0.77590,0.423936,0.8118
8,0.452885,0.78790,0.414349,0.8106
9,0.440796,0.79635,0.374689,0.8396


In [16]:
model.save_weights('weight_model.h5')

In [17]:
model.save('Model')

INFO:tensorflow:Assets written to: Model\assets


In [21]:
node_names = [node.name for node in tf.compat.v1.get_default_graph().as_graph_def().node]
node_names

[]

In [12]:
from tensorflow.python.tools import freeze_graph
from tensorflow.python.saved_model import tag_constants

In [20]:
freeze_graph.freeze_graph(
    input_graph=None,
    input_saver=None,
    input_binary=None,
    input_checkpoint=None,
    output_node_names='',
    restore_op_name=None,
    filename_tensor_name=None,
    output_graph= "frozen_graph.pb",
    clear_devices=None,
    initializer_nodes=None,
    input_saved_model_dir='Model',
    saved_model_tags=tag_constants.SERVING
)

ValueError: You need to supply the name of a node to --output_node_names.